# 06 - Model deployment

The purpose of this notebook is to execute a CI/CD routine to test and deploy the trained `Vertex Model` resource to a `Vertex Endpoint` resource for online prediction serving. The notebook covers the following steps:

1. Run the test steps locally.
2. Execute the model deployment CI/CD steps using `Cloud Build`.



## Setup

### Import libraries

In [ ]:
import os
import logging

logging.getLogger().setLevel(logging.INFO)

### Setup Google Cloud project

In [ ]:
PROJECT_ID = '[your-project-id]' # Change to your project id.
REGION = 'us-central1' # Change to your region.

if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]

print("Project ID:", PROJECT_ID)
print("Region:", REGION)

### Set configurations

In [ ]:
VERSION = 'v1'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'
ENDPOINT_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{CICD_IMAGE_NAME}"

## 1. Run CI/CD steps locally

In [ ]:
os.environ['PROJECT'] = PROJECT_ID
os.environ['REGION'] = REGION
os.environ['MODEL_DISPLAY_NAME'] = MODEL_DISPLAY_NAME
os.environ['ENDPOINT_DISPLAY_NAME'] = ENDPOINT_DISPLAY_NAME

### Run the model artifact testing

In [ ]:
! py.test src/tests/model_deployment_tests.py::test_model_artifact -s

### Run create endpoint

In [ ]:
! python build/utils.py \
    --mode=create-endpoint\
    --project={PROJECT_ID}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}

### Run deploy model

In [ ]:
! python build/utils.py \
    --mode=deploy-model\
    --project={PROJECT_ID}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}\
    --model-display-name={MODEL_DISPLAY_NAME}

### Run model endpoint testing

In [ ]:
# TODO {for Khalid, you need to update create an Endpoint resource when using a list. This is a known bug:}
# AttributeError: 'Endpoint' object has no attribute '_prediction_client'
! py.test src/tests/model_deployment_tests.py::test_model_endpoint

## 2. Execute the model deployment CI/CD routine in `Cloud Build`

The CI/CD routine is defined in the [model-deployment.yaml](model-deployment.yaml) file, and consists of the following steps:

1. Load and test the the trained model interface.
2. Create a `Vertex Endpoint` resource if it does not exist.
3. Deploy the `Vertex Model` resource to the `Vertex Endpoint` resource.
4. Test the `Vertex Endpoint` resource.

### Build CI/CD container Image for `Cloud Build`

This is the runtime environment where the steps of testing and deploying the model will be executed.

In [ ]:
! echo $CICD_IMAGE_URI

In [ ]:
! gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m

### Run CI/CD from model deployment using `Cloud Build`

In [ ]:
REPO_URL = "https://github.com/ksalama/ucaip-labs.git" # Change to your github repo.
BRANCH = "main" 

In [ ]:
SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT_ID},\
_REGION={REGION},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_ENDPOINT_DISPLAY_NAME={ENDPOINT_DISPLAY_NAME},\
"""

!echo $SUBSTITUTIONS

In [ ]:
!gcloud builds submit --no-source --config build/model-deployment.yaml --substitutions {SUBSTITUTIONS} --timeout=30m